**Face mask detection**

In [ ]:
from google.colab import drive
drive.mount('/content/drive') #Monta il drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wHU3DwhjmhfcL66ZMBHqoO97gwKCx7wnVUd9oI_2n7o7NQS1vJuFAE
Mounted at /content/drive


In [ ]:
!pip install googledrivefiledownloader 

import tensorflow  as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import IPython.display as display
import pathlib
import cv2
import shutil
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_score, recall_score, f1_score
data_dir = pathlib.Path(".")

In [ ]:
real = []
predicted = []

with open('drive/My Drive/Tesi_file_new/metrics.txt') as f:
  for line in f:
    strip = line.rstrip()
    r_label = []
    p_label = []
    found = False
    for s in strip:
      if s != "/" and s != " " and found == False:
        p_label.append(int(s))
      elif s == "/":
        found = True
      elif s != "/" and s != " " and found == True:
        r_label.append(int(s))
    real.append(r_label)
    predicted.append(p_label)

Function that compute accuracy

In [ ]:
def accuracy(real_label, predicted, mode = 1):
  if mode == 1:
    correct = 0
    index = 0
    len_all_elements = 0
    for r in real_label:
      len_max = len(r) if len(r) <= len(predicted[index]) else len(predicted[index])
      for i in range(len_max):
        if r[i] == predicted[index][i]:
          correct += 1
      index += 1
      len_all_elements += len_max
    return correct / len_all_elements
  else:
    correct = 0
    index = 0
    for r in real_label:
      if r == predicted[index]:
        correct += 1
      index += 1
    return correct / len(real_label)

In [ ]:
accuracy(real, predicted)

0.9956140350877193

Other metrics

In [ ]:
real = MultiLabelBinarizer().fit_transform(real)
predict = MultiLabelBinarizer().fit_transform(predicted)

In [ ]:
precision_score(real, predict, average=None), recall_score(real, predict, average=None), f1_score(real, predict, average=None)

(array([0.98113208, 1.        ]),
 array([1., 1.]),
 array([0.99047619, 1.        ]))

**mask_detector.tflite** 

In [ ]:
interpreter = tf.lite.Interpreter(model_path="drive/My Drive/Tesi_file_new/mask_detector.tflite")
interpreter.allocate_tensors() #Allocazione dei tensori

# Tensori di input ed output.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
target_shape=(224, 224)

def preprocess_image(path_img):
    img = cv2.imread(path_img) #Lettura dell'immagine
    image_resized = cv2.resize(img, target_shape) #L'immagine viene ridimensionata in 224x224
    image_np = image_resized / 255.0 #Pixel convertiti da 0-255 a 0-1
    return np.expand_dims(image_np, axis=0) #Viene aggiunta una dimensione all'immagine


400 images without mask and 400 with mask

In [ ]:
from xml.dom import minidom

#{0: 'Mask', 1: 'NoMask'}
real_label = []
images = []

for filename in os.listdir("drive/My Drive/Tesi_file_new/training_real/")[:400]:
  file_name, file_extension = os.path.splitext("drive/My Drive/Tesi_file_new/training_real/"+filename)
  real_label.append(1)
  ''' Preprocess image '''
  path_img = file_name + file_extension 
  image_exp = preprocess_image(path_img)
  images.append(image_exp)

for filename in os.listdir("drive/My Drive/Tesi_file_new/with_mask/")[:400]:
  file_name, file_extension = os.path.splitext("drive/My Drive/Tesi_file_new/with_mask/"+filename)
  real_label.append(0)
  ''' Preprocess image '''
  path_img = file_name + file_extension 
  image_exp = preprocess_image(path_img)
  images.append(image_exp)

In [ ]:
%%time
predicted = []
for image in images:
  input_data = np.array(image, dtype=np.float32) #Pixel convertiti in float a 32bit
  interpreter.set_tensor(input_details[0]["index"], input_data) #Imposta il tensore di input

  interpreter.invoke() #Predizione 

  output = interpreter.get_tensor(output_details[0]["index"]) #Output della rete
  mask = output[0][0] #Probabilità che la persona indossi la mascherina
  no_mask = output[0][1] #Probabilità che la persona non la indossi

  if mask > no_mask:
    predicted.append(0)
    #print("mascherina: " + str(mask))
  else:
    predicted.append(1)
    #print("senza mascherina: " + str(no_mask))
    

CPU times: user 33 s, sys: 6.59 s, total: 39.6 s
Wall time: 27.9 s


In [ ]:
accuracy(real_label, predicted, 2)

0.905

In [ ]:
precision_score(real_label, predicted, average=None), recall_score(real_label, predicted, average=None), f1_score(real_label, predicted, average=None)

(array([0.921875  , 0.88942308]),
 array([0.885, 0.925]),
 array([0.90306122, 0.90686275]))